## Training Model

### Loading packages

In [8]:
import numpy as np
import pandas as pd
import pdb

from deepiv.models import Treatment, Response
import deepiv.architectures as architectures
import deepiv.densities as densities

import tensorflow as tf

from keras.layers import Input, Dense
from keras.models import Model
from keras.layers.merge import Concatenate

### Importing data

In [16]:
expression_levels = pd.read_csv('/Users/Walker/Desktop/Winter/CS230/deepiv-gwas/derived_data/expression_levels.csv')
gene_variants = pd.read_csv('/Users/Walker/Desktop/Winter/CS230/deepiv-gwas/derived_data/gene_variants.csv')
outcomes = pd.read_csv('/Users/Walker/Desktop/Winter/CS230/deepiv-gwas/derived_data/outcomes.csv')

In [23]:
# sanity check
assert(gene_variants.shape[1] - 1 == outcomes.shape[0])
print("Dim(z) = ", gene_variants.shape)
print("Dim(p) = ", expression_levels.shape)
print("Dim(y) = ", outcomes.shape)

Dim(z) =  (13980, 12667)
Dim(p) =  (53196, 12667)
Dim(y) =  (12666, 2)


In [36]:
expression_levels = expression_levels.drop('mrna', axis = 1).transpose().as_matrix()
gene_variants = gene_variants.drop('hugo', axis = 1).transpose().as_matrix()
outcomes = outcomes['outcome'].as_matrix()

ValueError: labels ['mrna'] not contained in axis

### Training first stage model

In [ ]:
n = len(outcomes)
epochs = int(1500000./float(n)) # heuristic to get epochs
batch_size = 100

instruments = Input(shape=(gene_variants.shape[1],), name = "instruments")
hidden = [21000, 35000, 48000]

activation = "relu"

n_components = expression_levels.shape[1]

est_treat = architectures.feed_forward_net(instruments,
                                          lambda x: densities.mixture_of_gaussian_output(x, n_components),
                                          hidden_layers = hidden,
                                          activations = activation)

treatment_model = Treatment(inputs=[instruments], outputs=est_treat)
print("Fitting treatment")
treatment_model.compile('adam',
                       loss='mixture_of_gaussians',
                       n_components = n_components,
                       verbose = 1)

treatment_model.fit(gene_variants, expression_levels,
                   epochs = epochs,
                   batch_size = batch_size)

Fitting treatment
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/118


13980